# Chapter 1. Probability
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap01.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
import CSV as Csv
import DataFrames as Dfs
import Statistics as Stats

## Functionality developed in this chapter

In [ ]:
function getProb(v::BitVector)::Float64
    return Stats.mean(v)
end

In [ ]:
function getCondProb(
    proposition::BitVector, given::BitVector
    )::Float64
    return getProb(proposition[given])
end

## Linda the Banker

> Linda is 31 years old, single, outspoken, and very bright. She majored in philosophy. As a student, she was deeply concerned with issues of discrimination and social justice, and also participated in anti-nuclear demonstrations. Which is more probable?
> 1. Linda is a bank teller.
> 2. Linda is a bank teller and is active in the feminist movement.

## Probability

[Link to the data file (gss_bayes.csv) online](https://raw.githubusercontent.com/AllenDowney/BiteSizeBayes/master/gss_bayes.csv)

In [ ]:
gss = Dfs.DataFrame(Csv.File("./gss_bayes.csv"))
first(gss, 5)

## Fraction of bankers

The code for “Banking and related activities” is 6870, so we can select bankers like this:

In [ ]:
banker = (gss.indus10 .== 6870)
first(banker, 5)

In [ ]:
sum(banker)

In [ ]:
Stats.mean(banker)

## The probability function

<pre>
function getProb(v::BitVector)::Float64
    return Stats.mean(v)
end
</pre>

In [ ]:
getProb(banker)

columng `gss.sex` coding:
1. male
2. female

In [ ]:
female = (gss.sex .== 2);

In [ ]:
getProb(female)

## Political Views and Parties

`polviews` coding:
1. Extremely liberal
2. Liberal
3. Slightly liberal
4. Moderate
5. Slightly conservative
6. Conservative
7. Extremely conservative

In [ ]:
liberal = (gss.polviews .<= 3);

In [ ]:
getProb(liberal)

`partyid` coding:
0. Strong democrat
1. Not strong democrat
2. Independent, near democrat
3. Independent
4. Independent, near republican
5. Not strong republican
6. Strong republican
7. Other party

In [ ]:
democrat = (gss.partyid .<= 1);

In [ ]:
getProb(democrat)

## Conjunction

“Conjunction” is another name for the logical `and` operation. If you have two propositions, `A` and `B`, the conjunction `A` and `B` is `True` if both `A` and `B` are `True`, and `False` otherwise.

In [ ]:
getProb(banker)

In [ ]:
getProb(democrat)

In [ ]:
getProb(banker .& democrat)

As we should expect, `getProb(banker .& democrat)` is less than `getProb(banker)`, because not all bankers are Democrats.

We expect conjunction to be commutative; that is, `A & B` should be the same as `B & A`. To check, we can also compute `getProb(democrat & banker)`:

In [ ]:
getProb(democrat .& banker)

## Conditional Probability

Questions:
- What is the probability that a respondent is a Democrat, given that they are liberal?
- What is the probability that a respondent is female, given that they are a banker?
- What is the probability that a respondent is liberal, given that they are female?

In [ ]:
democratGivenLiberal = democrat[liberal];

In [ ]:
getProb(democratGivenLiberal)

In [ ]:
femaleGivenBanker = female[banker]
getProb(femaleGivenBanker)

In [ ]:
getCondProb(liberal, female)

## Conditional Probability is Not Commutative

In [ ]:
getCondProb(female, banker)

About 77% of bankers are female

In [ ]:
getCondProb(banker, female)

About 2% of female are bankers